In [1]:
'''PREFERENCES'''

#Input file name or naming pattern to search for here
targetFileName = 'data_0.5_2.6_000160'
# Number of test repetitions
loop_cnt = 1
#Whether you want the graph visualized or not
graph_visualization = False

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath("../src"))
from py_graspi import descriptors as ds
from py_graspi import graph as ig
import time
import tracemalloc

data_path = os.path.abspath("../data/2phase/2D-morphologies/data/") #Adjust data path as necessary
expected_results_path = os.path.abspath("expected_results/") #Adjust expected results path as necessary
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

for test_file in test_files:
    if targetFileName not in test_file:
        continue
    total_graph_time = 0
    for i in range(loop_cnt):
        tracemalloc.start()
        graph_start = time.time()
        file_path = os.path.join(data_path, test_file + ".txt")
        graph_data = ig.generateGraph(file_path,True)
        g = graph_data.graph
        _stats = tracemalloc.get_traced_memory()
        graph_end = time.time()
        tracemalloc.stop()
        graph_mem = _stats[1]-_stats[0]
        stats = ds.compute_descriptors(graph_data, file_path)
        total_graph_time += graph_end - graph_start
        if graph_visualization:
            ig.visualize(graph_data.graph, graph_data.is_2D)

    print(f"{test_file} Results")

    expected_log = os.path.join(expected_results_path, f"{test_file}.log")
    if not os.path.exists(expected_log):
        print(f"Expected result log file not found: {expected_log}")
        continue

    with open(expected_log) as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    descriptor_time = stats["time"]
    descriptor_mem = stats["mem"]

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = total_graph_time/loop_cnt
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



data_0.5_2.6_000160 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_wf_D failed - 0.3165592285854695 is not the same as expected 0.313409
ABS_f_D passed
DISS_wf10_D failed - 0.3833910029546846 is not the same as expected 0.39619
DISS_f10_D failed - 0.7711583221908582 is not the same as expected 0.772694
CT_f_e_conn passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_e_conn passed
CT_e_D_An passed
CT_e_A_Ca passed
CT_f_D_tort1 passed
CT_f_A_tort1 passed
CT_n_D_adj_An passed
CT_n_A_adj_Ca passed
Total time to calculate graph: 32.02923107147217 second(s)
Total time to calculate descriptors: 10.84945297241211 second(s)
Peak memory usage for graph generation: 37032550 bytes
Peak memory usage for descriptor calculation: 55606511 bytes
{'STAT_n': 40501, 'STAT_e': 2022, 'STAT_n_D': 20193, 'STAT_n_A': 20308, 'STAT_CC_D': 8, 'STAT_CC_A': 6, 'STAT_CC_D_An': 4, 'STAT_CC_A_Ca': 3, 'ABS_wf_D': 0.31655